<h1>Dependencies</h1>

In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

<h1>Embed Class Slides and Textbooks (only do this if embedding new documents)</h1>

In [2]:
# Load documents
loader = DirectoryLoader('../docs', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3.2", base_url="http://localhost:11434")
vectorstore = Chroma.from_documents(persist_directory='../chroma_v2', documents=splits, embedding=embeddings)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong poi

<h1>Initialize Llama LLM locally</h1>

In [ ]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
# Initialize LLama 3.2
llm = Ollama(model="llama3.2", base_url="http://localhost:11434")



<h1>Generate prompt with GUI using gradio</h1>

In [ ]:
import gradio as gr

def answer(message: str, history: list[str]):
    print(f'-----------------------History: {history}')
    #Use llm to determine if question is relevant to network security
    relevant_response = llm(f"Question: Is this question: {message} relevant to network security? Simple yes or no. Don't overthink it. Obvious answers only.").lower()
    print(relevant_response)
    relevant = 'yes' in relevant_response

    #Find similar docs if relevant
    similar_docs = vectorstore.similarity_search(message, k=10) if relevant else []

    if relevant:         
        citations = "\n".join([ f"Rank {i+1}: {doc.metadata['source']} (Page {doc.metadata.get('page', 'Unknown')})" for i, doc in enumerate(similar_docs)])        
        context = "\n".join([doc.page_content for doc in similar_docs]) 
    else: 
        citations = "Source: Not relevant to network security."
        context = ""
 

    #Document the source
    source = "\n".join([f"Source: {document.metadata['source'][document.metadata['source'].index('/')+1:]} Page Number: {document.metadata['page']}" for document in similar_docs]) if relevant else "Source: Not relevant to network security."

    #Construct prompt using history from past requests - this will allows to take interactive quizzes
    prompt = "\n".join([f"User: {item[0]}\nBot: {item[1][:item[1].index('Source')]}" for item in history]) + f"\nUser: {message}"
    
    #Construct context if relevant
    context = "\n".join([doc.page_content for doc in similar_docs]) if relevant else []

    #Send constructed prompt to llm and add message, result to history
    response = llm(f"Context: {context}\nQuestion: {prompt}") if relevant else llm(f"Question: {prompt}")
    #history.append([message, response])

    #Display the reponse as well as the source
    return (response + "\n\n" + citations)


chatbot = gr.ChatInterface(
    fn=answer,
    title="Group 5 Quizbot",
    description="Enter the following prompt to be quizzed: Generate a multiple choice, true/false, or short answer network security question and provide feedback to my answer."
)

chatbot.launch(server_port=7860)
